<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

<h2>Installing and importing libraries</h2>

In [1]:
!pip install beautifulsoup4
!pip install lxml
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

print("Everything installed :)")

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Everything installed :)


In [2]:
from bs4 import BeautifulSoup # Library for web scraping
import requests               # Library to handle requests
import numpy as np            # library to handle data in a vectorized manner
import pandas as pd           # Library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#import json # library to handle JSON files
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from IPython.display import display_html

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported ;)")

Libraries imported ;)


<h2>1. Create initial neighborhoods dataframe</h2>

In [3]:
# Scraping wiki and getting table

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')
print(soup.title)
tab = str(soup.table)
#display_html(tab,raw=True)

<title>List of postal codes of Canada: M - Wikipedia</title>


In [4]:
# Converting table to dataframe

df_tor = pd.read_html(tab)
df_tor = df_tor[0]
#df_tor.head()

In [5]:
# Renaming columns

df_tor = df_tor.rename(columns={'Postal Code': 'PostalCode', 'Neighbourhood':'Neighborhood'})
#df_tor.head()

In [6]:
# Dropping 'Not assigned' boroughs

indexNames = df_tor[ df_tor['Borough'] =='Not assigned'].index
df_tor.drop(indexNames , inplace=True)
#df_tor.head(10)

In [7]:
# Assigning the same name from Borough where Neighborhood is 'Not assigned'

df_tor.loc[df_tor['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df_tor['Borough']
#df_tor.head()

In [8]:
# Resetting index

df_tor.reset_index(inplace=True)
df_tor.drop('index', axis=1, inplace=True)
df_tor.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
df_tor.shape

(103, 3)

<h2>2. Create neighborhoods with coordinates dataframe</h2>

In [10]:
# Downloading coordinates for postal codes 

df_coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
#df_coordinates.head()

In [11]:
# Renaming column

df_coordinates = df_coordinates.rename(columns={'Postal Code': 'PostalCode'})
#df_coordinates.head()

In [12]:
# Merging the two dataframes

df_neighborhoods = pd.merge(df_tor, df_coordinates, on = 'PostalCode')
df_neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<h2>3. Clustering and analyzing data from neighborhoods</h2>

In [13]:
# Getting the geospatial data for Toronto

address = 'Toronto, ON'
geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [14]:
# Creating a map of Toronto with the neighborhoods marked

# Create map using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# Add markers to map
for lat, lng, borough, neighborhood in zip(df_neighborhoods['Latitude'], df_neighborhoods['Longitude'], df_neighborhoods['Borough'], df_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h4>If the map is not visible on github, check the read me file to see it</h4>

In [15]:
# Defining the Foursquare credentials and version for the API calls

CLIENT_ID = 'DXCAVL4WFDOCH3AKF2JWJ44GAP4YI5O0DSQJCGAS14I21203' # your Foursquare ID
CLIENT_SECRET = 'TO5NBHA0CTDXXRXTBYC2IT5UXVYYV40U4ME1V21Q3FEWX3EV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DXCAVL4WFDOCH3AKF2JWJ44GAP4YI5O0DSQJCGAS14I21203
CLIENT_SECRET:TO5NBHA0CTDXXRXTBYC2IT5UXVYYV40U4ME1V21Q3FEWX3EV


In [16]:
# Function to explore venues in each neighborhood

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)      
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
# Getting the data for toronto venues

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
toronto_venues = getNearbyVenues(names=df_neighborhoods['Neighborhood'],
                                   latitudes=df_neighborhoods['Latitude'],
                                   longitudes=df_neighborhoods['Longitude']
                                  )

In [18]:
print("Shape of Toronto Venues dataframe: " + str(toronto_venues.shape))
#toronto_venues.head()
#toronto_venues.groupby('Neighborhood').count()
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

Shape of Toronto Venues dataframe: (2146, 7)
There are 270 unique categories.


In [19]:
# Create dataframe for venues in neighborhoods

# One hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# Add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# Move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

#toronto_onehot.head()
toronto_onehot.shape

(2146, 270)

In [20]:
# Group dataframe by neighborhood by taking the mean of frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
#toronto_grouped.head()
toronto_grouped.shape

(96, 270)

In [21]:
# Function to get a dataframe for the most common venues for each neighborhood

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
# Get the most common venues for each Toronto neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

#neighborhoods_venues_sorted.head()

<h4>Clustering</h4>

In [23]:
# Set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [24]:
# Create new dataframes that includes the cluster and top 10 venues

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged=toronto_merged.dropna()
#toronto_merged.head() # check the last columns!

In [25]:
# Finally, we will visualize the clusters on the map

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h4>If the map is not visible on github, check the read me file to see it</h4>

<h4>Let's check the clusters formed </h4>

In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,0.0,Fast Food Restaurant,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant


In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1.0,French Restaurant,Hockey Arena,Coffee Shop,Portuguese Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,Downtown Toronto,1.0,Coffee Shop,Pub,Bakery,Park,Restaurant,Breakfast Spot,Café,Theater,Gym / Fitness Center,Farmers Market
3,North York,1.0,Clothing Store,Miscellaneous Shop,Accessories Store,Event Space,Boutique,Vietnamese Restaurant,Gift Shop,Coffee Shop,Furniture / Home Store,Athletics & Sports
4,Downtown Toronto,1.0,Coffee Shop,Diner,Yoga Studio,College Auditorium,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café
7,North York,1.0,Gym,Beer Store,Coffee Shop,Japanese Restaurant,Restaurant,Asian Restaurant,Baseball Field,Café,Dim Sum Restaurant,Discount Store


In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,2.0,Home Service,Bar,Women's Store,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
62,Central Toronto,2.0,Garden,Home Service,Women's Store,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center


In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,3.0,Park,Food & Drink Shop,Pool,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center
21,York,3.0,Park,Pool,Women's Store,Golf Course,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
35,East York,3.0,Park,Convenience Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dance Studio,Dog Run
52,North York,3.0,Park,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
61,Central Toronto,3.0,Park,Swim School,Bus Line,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Deli / Bodega,Distribution Center


In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,East York,4.0,Pizza Place,Gym / Fitness Center,Bank,Breakfast Spot,Pet Store,Pharmacy,Intersection,Gastropub,Athletics & Sports,Diner
10,North York,4.0,Pizza Place,Park,Sushi Restaurant,Pub,Japanese Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Curling Ice
14,East York,4.0,Dance Studio,Curling Ice,Park,Skating Rink,Spa,Video Store,Pharmacy,Beer Store,Athletics & Sports,Discount Store
50,North York,4.0,Gym,Furniture / Home Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dance Studio,Distribution Center
56,York,4.0,Turkish Restaurant,Sandwich Place,Skating Rink,Discount Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Women's Store
